# **NLP2_Run3**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 831.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 958.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 816.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.7 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be us

In [2]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [4]:
problem = """A buyer needs to acquire 239,600,480 units of a product and is considering bids from five suppliers, labeled A through E, each of whom can only supply a portion of the total required amount.
Each vendor has proposed different pricing structures, incorporating both setup fees and variable unit costs that change based on the quantity ordered.

The buyer's objective is to allocate the order among these suppliers to minimize overall costs, accounting for both setup and unit costs.

Vendor A offers a set up cost of $3855.34 and a unit cost of $61.150 per thousand of units.
Vendor A can supply up to 33 million units.

Vendor B offers a set up cost of $125,804.84 if purchasing between 22,000,000-70,000,000 units from vendor B with a unit cost of $68.099 per thousand units.
If purchasing between 70,000,001-100,000,000 units from vendor B, the set up cost increases to $269304.84 and the unit cost sinks to $66.049 per thousand units.
If purchasing between 100,000,001-150,000,000 units from vendor B, the unit cost per thousand units further decreases to $64.099, but the set up cost increases to $464304.84.
If purchasing between 150,000,001 and 160,000,000 units from vendor B, the unit cost is $62.119 per thousand units and the set up cost equals $761304.84.

Vendor C offers set up costs of $13,456.00 and a unit cost of $62.019 per thousand units.
Vendor C can supply up to 165.6 million units. Vendor D offers set up costs of $6,583.98 and a unit cost of $72.488 for a set of thousand units.

Vendor D can supply up to 12 million units at a price of $72.488 per thousand units and with a set up cost of $6583.98.

Vendor E offers free set up if purchasing between 0 and 42 million units of vendor E with a unit price of $70.150 per thousand units.
If purchasing between 42,000,001 and 77 million units from vendor E, the unit cost starts at $68.150 per thousand units, but with every one million units purchased the price decreases at a rate of 0.05 percent. An additional set up cost of $84000 will be charged as well.

Note that zero units may be purchased from vendor B: otherwise no positive number of units less than 22,000,000 may be purchased."""

In [5]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 3
)


In [6]:
print(response1.choices[0].message.content)

Let A, B1, B2, B3, B4, C, D, E1 and E2 be the number of units bought from the respective suppliers. 

A represents units purchased from vendor A, 
B1 represents units purchased from vendor B in the range 22,000,000-70,000,000 units,
B2 represents units purchased from vendor B in the range 70,000,001-100,000,000 units,
B3 represents units purchased from vendor B in the range 100,000,001-150,000,000 units,
B4 represents units purchased from vendor B in the range 150,000,001-160,000,000 units,
C represents units purchased from vendor C, 
D represents units purchased from vendor D, 
E1 represents units purchased from vendor E in the range 0-42,000,000 units,
E2 represents units purchased from vendor E in the range 42,000,001-77,000,000 units.


In [7]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [8]:
print(response2.choices[0].message.content)

Minimize Total Cost = 3855.34 + 61.150 * A * 0.001 + (125804.84 + 68.099 * B1 * 0.001) + (269304.84 + 66.049 * B2 * 0.001) + (464304.84 + 64.099 * B3 * 0.001) + (761304.84 + 62.119 * B4 * 0.001) + 13456 + 62.019 * C * 0.001 + 6583.98 + 72.488 * D * 0.001 + 70.150 * E1 * 0.001 + (84000 + 68.150 * (1 - (E2 - 42 * 10^6) * 0.05 * 0.01) * E2 * 0.001)


In [9]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [10]:
print(response3.choices[0].message.content)

1. Units constraints:

   A ≥ 0, 
   A ≤ 33,000,000,
   
   B1 ≥ 0,
   B1 ≤ 48,000,000,  (70,000,000 - 22,000,000)

   B2 ≥ 0,
   B2 ≤ 30,000,000,
   
   B3 ≥ 0,
   B3 ≤ 50,000,000,
   
   B4 ≥ 0,
   B4 ≤ 10,000,000,
   
   C ≥ 0,
   C ≤ 165,600,000,
   
   D ≥ 0,
   D ≤ 12,000,000,
   
   E1 ≥ 0,
   E1 ≤ 42,000,000,
   
   E2 ≥ 0,
   E2 ≤ 35,000,000, (77,000,000 - 42,000,000)

2. Supply fulfillment constraint: 

   A + B1 + B2 + B3 + B4 + C + D + E1 + E2 = 239,600,480

3. No partial orders from vendor B:

   If B1, B2, B3, B4 > 0
   then B1+B2+B3+B4 ≥ 22,000,000

4. Integers only:

   Binary constraint: B1, B2, B3, B4 can take only integer values of millions or zeroes.


### **Generate Pyomo Code**

In [11]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [12]:
Markdown(response4.choices[0].message.content)

Below is a Pyomo code, e.g. for a Minimization optimization problem that fits the problem described above:

```python
from pyomo.environ import *

# Instantiate a Concrete Model to represent the problem
model = ConcreteModel()

# Decision Variables
model.A = Var(domain=NonNegativeReals)
model.B1 = Var(domain=NonNegativeIntegers)
model.B2 = Var(domain=NonNegativeIntegers)
model.B3 = Var(domain=NonNegativeIntegers)
model.B4 = Var(domain=NonNegativeIntegers)
model.C = Var(domain=NonNegativeReals)
model.D = Var(domain=NonNegativeReals)
model.E1 = Var(domain=NonNegativeReals)
model.E2 = Var(domain=NonNegativeReals)

# Objective Function
model.obj = Objective(expr = model.A*61.150 + model.B1*68.099 + model.B2*66.049 + model.B3*64.099 + model.B4*62.119 + model.C*62.019 + model.D*72.488 + model.E1*70.150 + model.E2*68.150*(1 - 0.05*(model.E2 - 42*(10**6))*0.01),
                      sense = minimize)

# Constraints
model.cons1 = Constraint(expr = model.A <= 33*(10**6))
model.cons2 = Constraint(expr = model.B1 <= 48*(10**6))
model.cons3 = Constraint(expr = model.B2 <= 30*(10**6))
model.cons4 = Constraint(expr = model.B3 <= 50*(10**6))
model.cons5 = Constraint(expr = model.B4 <= 10*(10**6))
model.cons6 = Constraint(expr = model.C <= 165.6*(10**6))
model.cons7 = Constraint(expr = model.D <= 12*(10**6))
model.cons8 = Constraint(expr = model.E1 <= 42*(10**6))
model.cons9 = Constraint(expr = model.E2 <= 35*(10**6))
model.cons10 = Constraint(expr = model.A + model.B1 + model.B2 + model.B3 + model.B4 + model.C + model.D + model.E1 + model.E2 == 239.6*(10**6))
model.cons11 = Constraint(expr = (model.B1 + model.B2 + model.B3 + model.B4) >= 22*(10**6), rule = lambda model : sum(model.B[i] for i in range(1,5)) > 0)

#Solve the Model
SolverFactory('gurobi').solve(model).write()
```
Please note that you might have to process the solution from solver to get in terms of the original variables and also this is a generic code which can be modified as per actual constraints and requirement.

### **Run the code Generated by GPT4**

In [13]:
# Install Python API for AMPL
!pip install amplpy --upgrade

from amplpy import AMPL,ampl_notebook

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 18.1 MB/s eta 0:00:00


In [15]:
_ = load_dotenv(find_dotenv('second_knitro_license.env'))
my_knitro_license = os.environ['knitro_license']

ampl = ampl_notebook(
    modules=["coin"],  # modules to install
    license_uuid=my_knitro_license,  # license to use
)

Licensed to AMPL Community Edition License for <cevikmelis14@gmail.com>.


In [16]:
from pyomo.environ import *

# Instantiate a Concrete Model to represent the problem
model = ConcreteModel()

# Decision Variables
model.A = Var(domain=NonNegativeReals)
model.B1 = Var(domain=NonNegativeIntegers)
model.B2 = Var(domain=NonNegativeIntegers)
model.B3 = Var(domain=NonNegativeIntegers)
model.B4 = Var(domain=NonNegativeIntegers)
model.C = Var(domain=NonNegativeReals)
model.D = Var(domain=NonNegativeReals)
model.E1 = Var(domain=NonNegativeReals)
model.E2 = Var(domain=NonNegativeReals)

# Objective Function
model.obj = Objective(expr = model.A*61.150 + model.B1*68.099 + model.B2*66.049 + model.B3*64.099 + model.B4*62.119 + model.C*62.019 + model.D*72.488 + model.E1*70.150 + model.E2*68.150*(1 - 0.05*(model.E2 - 42*(10**6))*0.01),
                      sense = minimize)

# Constraints
model.cons1 = Constraint(expr = model.A <= 33*(10**6))
model.cons2 = Constraint(expr = model.B1 <= 48*(10**6))
model.cons3 = Constraint(expr = model.B2 <= 30*(10**6))
model.cons4 = Constraint(expr = model.B3 <= 50*(10**6))
model.cons5 = Constraint(expr = model.B4 <= 10*(10**6))
model.cons6 = Constraint(expr = model.C <= 165.6*(10**6))
model.cons7 = Constraint(expr = model.D <= 12*(10**6))
model.cons8 = Constraint(expr = model.E1 <= 42*(10**6))
model.cons9 = Constraint(expr = model.E2 <= 35*(10**6))
model.cons10 = Constraint(expr = model.A + model.B1 + model.B2 + model.B3 + model.B4 + model.C + model.D + model.E1 + model.E2 == 239.6*(10**6))
model.cons11 = Constraint(expr = (model.B1 + model.B2 + model.B3 + model.B4) >= 22*(10**6), rule = lambda model : sum(model.B[i] for i in range(1,5)) > 0)

# Solve the Model
SolverFactory('couenne').solve(model).write()

ValueError: Duplicate initialization: Constraint() only accepts one of 'rule=' 'expr='

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**

In [25]:
from pyomo.environ import *

# Instantiate a Concrete Model to represent the problem
model = ConcreteModel()

# Decision Variables
model.A = Var(domain=NonNegativeReals)
model.B1 = Var(domain=NonNegativeIntegers)
model.B2 = Var(domain=NonNegativeIntegers)
model.B3 = Var(domain=NonNegativeIntegers)
model.B4 = Var(domain=NonNegativeIntegers)
model.C = Var(domain=NonNegativeReals)
model.D = Var(domain=NonNegativeReals)
model.E1 = Var(domain=NonNegativeReals)
model.E2 = Var(domain=NonNegativeReals)

# Objective Function
model.obj = Objective(expr = 3855.34 + model.A*61.150*0.001 + (125804.84 + model.B1*68.099*0.001) + (269304.84+model.B2*66.049*0.001) + (464304.84+model.B3*64.099*0.001) + (761304.84+model.B4*62.119*0.001) + 13456 + model.C*62.019*0.001 + 6583.98 + model.D*72.488*0.001 + model.E1*70.150*0.001 + (84000 + 68.150*(1 - 0.01*0.05*(model.E2 - 42*(10**6))*model.E2*0.001)),
                      sense = minimize)

# Constraints
model.cons1 = Constraint(expr = model.A <= 33*(10**6))
model.cons2 = Constraint(expr = model.B1 <= 48*(10**6))
model.cons3 = Constraint(expr = model.B2 <= 30*(10**6))
model.cons4 = Constraint(expr = model.B3 <= 50*(10**6))
model.cons5 = Constraint(expr = model.B4 <= 10*(10**6))
model.cons6 = Constraint(expr = model.C <= 165.6*(10**6))
model.cons7 = Constraint(expr = model.D <= 12*(10**6))
model.cons8 = Constraint(expr = model.E1 <= 42*(10**6))
model.cons9 = Constraint(expr = model.E2 <= 35*(10**6))
model.cons10 = Constraint(expr = model.A + model.B1 + model.B2 + model.B3 + model.B4 + model.C + model.D + model.E1 + model.E2 == 239600480)
model.cons11 = Constraint(expr = (model.B1 + model.B2 + model.B3 + model.B4) >= 22*(10**6))


# Solve the Model
SolverFactory('couenne').solve(model).write()
print(model.obj())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 9
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: couenne\x3a Optimal
  Termination condition: optimal
  Id: 3
  Error rc: 0
  Time: 0.2899928092956543
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0
16625268.997520002
